In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import yfinance as yf
import uuid


In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
keyspace_name='info_stock'
session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)

In [4]:
session.set_keyspace('info_stock')

In [5]:
session.execute("""
CREATE TABLE IF NOT EXISTS financial_data (
    ticker TEXT PRIMARY KEY,
    name TEXT,
    sector TEXT,
    industry TEXT,
    total_debt DOUBLE,
    total_assets DOUBLE,
    total_revenue DOUBLE,
    interest_income DOUBLE,
    long_business_summary TEXT
)
""")

In [7]:

# ✅ เตรียม Insert Statement นอกลูป
insert_stmt = session.prepare("""
INSERT INTO financial_data (
    ticker, name, sector, industry,
    total_debt, total_assets, total_revenue,
    interest_income, long_business_summary
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

# ✅ โหลดรายชื่อหุ้น
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().astype(str).tolist()
    print(f"✅ Loaded {len(symbols)} symbols from Excel.")
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# 🔧 แปลงเป็น ticker แบบที่ใช้กับ yfinance
tickers = [symbol.strip().upper() + ".BK" for symbol in symbols]

# 🔁 ดึงและโหลดข้อมูลทีละตัว
for ticker in tickers:
    try:
        print(f"📥 Fetching {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info

        # 🔍 ดึงข้อมูลที่จำเป็น
        name = info.get('shortName', '')
        sector = info.get('sector', '')
        industry = info.get('industry', '')
        summary = info.get('longBusinessSummary', '')

        total_debt = float(info.get('totalDebt', 0.0) or 0.0)
        total_assets = float(info.get('totalAssets', 1.0) or 1.0)
        total_revenue = float(info.get('totalRevenue', 0.0) or 0.0)

        # 🔸 ประเมิน interest income จาก EBITDA margin
        ebitda_margin = float(info.get('ebitdaMargins', 0.0) or 0.0)
        interest_income = ebitda_margin * total_assets

        # ✅ Insert ลง Cassandra
        session.execute(insert_stmt, (
            ticker, name, sector, industry,
            total_debt, total_assets, total_revenue,
            interest_income, summary
        ))

        print(f"✅ Inserted {ticker} successfully.")

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")


✅ Loaded 923 symbols from Excel.
📥 Fetching 24CS.BK...
✅ Inserted 24CS.BK successfully.
📥 Fetching 2S.BK...
✅ Inserted 2S.BK successfully.
📥 Fetching 3BBIF.BK...
✅ Inserted 3BBIF.BK successfully.
📥 Fetching A.BK...
✅ Inserted A.BK successfully.
📥 Fetching A5.BK...
✅ Inserted A5.BK successfully.
📥 Fetching AAI.BK...
✅ Inserted AAI.BK successfully.
📥 Fetching AAV.BK...
✅ Inserted AAV.BK successfully.
📥 Fetching ABM.BK...
✅ Inserted ABM.BK successfully.
📥 Fetching ACAP.BK...
✅ Inserted ACAP.BK successfully.
📥 Fetching ACC.BK...
✅ Inserted ACC.BK successfully.
📥 Fetching ACE.BK...
✅ Inserted ACE.BK successfully.
📥 Fetching ACG.BK...
✅ Inserted ACG.BK successfully.
📥 Fetching ADB.BK...
✅ Inserted ADB.BK successfully.
📥 Fetching ADD.BK...
✅ Inserted ADD.BK successfully.
📥 Fetching ADVANC.BK...
✅ Inserted ADVANC.BK successfully.
📥 Fetching ADVICE.BK...
✅ Inserted ADVICE.BK successfully.
📥 Fetching AE.BK...
✅ Inserted AE.BK successfully.
📥 Fetching AEONTS.BK...
✅ Inserted AEONTS.BK successfull

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GL.BK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GL.BK&crumb=z8BcY3ZrUyJ


❌ Error with GL.BK: 'NoneType' object has no attribute 'update'
📥 Fetching GLAND.BK...
✅ Inserted GLAND.BK successfully.
📥 Fetching GLOBAL.BK...
✅ Inserted GLOBAL.BK successfully.
📥 Fetching GLOCON.BK...
✅ Inserted GLOCON.BK successfully.
📥 Fetching GLORY.BK...
✅ Inserted GLORY.BK successfully.
📥 Fetching GPI.BK...
✅ Inserted GPI.BK successfully.
📥 Fetching GPSC.BK...
✅ Inserted GPSC.BK successfully.
📥 Fetching GRAMMY.BK...
✅ Inserted GRAMMY.BK successfully.
📥 Fetching GRAND.BK...
✅ Inserted GRAND.BK successfully.
📥 Fetching GREEN.BK...
✅ Inserted GREEN.BK successfully.
📥 Fetching GROREIT.BK...
✅ Inserted GROREIT.BK successfully.
📥 Fetching GSTEEL.BK...
✅ Inserted GSTEEL.BK successfully.
📥 Fetching GTB.BK...
✅ Inserted GTB.BK successfully.
📥 Fetching GTV.BK...
✅ Inserted GTV.BK successfully.
📥 Fetching GULFI.BK...
✅ Inserted GULFI.BK successfully.
📥 Fetching GUNKUL.BK...
✅ Inserted GUNKUL.BK successfully.
📥 Fetching GVREIT.BK...
✅ Inserted GVREIT.BK successfully.
📥 Fetching GYT.BK...
✅

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/INTUCH.BK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=INTUCH.BK&crumb=z8BcY3ZrUyJ


❌ Error with INTUCH.BK: 'NoneType' object has no attribute 'update'
📥 Fetching IP.BK...
✅ Inserted IP.BK successfully.
📥 Fetching IRC.BK...
✅ Inserted IRC.BK successfully.
📥 Fetching IRCP.BK...
✅ Inserted IRCP.BK successfully.
📥 Fetching IROYAL.BK...
✅ Inserted IROYAL.BK successfully.
📥 Fetching IRPC.BK...
✅ Inserted IRPC.BK successfully.
📥 Fetching ISSARA.BK...
✅ Inserted ISSARA.BK successfully.
📥 Fetching IT.BK...
✅ Inserted IT.BK successfully.
📥 Fetching ITC.BK...
✅ Inserted ITC.BK successfully.
📥 Fetching ITD.BK...
✅ Inserted ITD.BK successfully.
📥 Fetching ITEL.BK...
✅ Inserted ITEL.BK successfully.
📥 Fetching ITNS.BK...
✅ Inserted ITNS.BK successfully.
📥 Fetching ITTHI.BK...
✅ Inserted ITTHI.BK successfully.
📥 Fetching IVF.BK...
✅ Inserted IVF.BK successfully.
📥 Fetching IVL.BK...
✅ Inserted IVL.BK successfully.
📥 Fetching J.BK...
✅ Inserted J.BK successfully.
📥 Fetching JAK.BK...
✅ Inserted JAK.BK successfully.
📥 Fetching JAS.BK...
✅ Inserted JAS.BK successfully.
📥 Fetching JCK.

In [ ]:
# 🔁 ดึงและโหลดทีละตัว
for ticker in tickers:
    try:
        print(f"📥 Fetching {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info

        # 🔍 ดึงข้อมูลที่จำเป็น
        name = info.get('shortName', '')
        sector = info.get('sector', '')
        industry = info.get('industry', '')
        summary = info.get('longBusinessSummary', '')

        total_debt = float(info.get('totalDebt', 0.0) or 0.0)
        total_assets = float(info.get('totalAssets', 1.0) or 1.0)
        total_revenue = float(info.get('totalRevenue', 0.0) or 0.0)

        # interest income ไม่มีใน yfinance → ประเมินจาก EBITDA Margin
        ebitda_margin = float(info.get('ebitdaMargins', 0.0) or 0.0)
        interest_income = ebitda_margin * total_assets
        insert_stmt = session.prepare("""
INSERT INTO financial_data (
    ticker, name, sector, industry,
    total_debt, total_assets, total_revenue,
    interest_income, long_business_summary
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")
        # ✅ Insert ลง Cassandra
        session.execute(insert_stmt, (
            ticker, name, sector, industry,
            total_debt, total_assets, total_revenue,
            interest_income, summary
        ))

        print(f"✅ {ticker} inserted.")

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

📥 Fetching PTT.BK...
✅ PTT.BK inserted.
📥 Fetching CPALL.BK...
✅ CPALL.BK inserted.
📥 Fetching KBANK.BK...
✅ KBANK.BK inserted.
📥 Fetching AOT.BK...
✅ AOT.BK inserted.
